# 清洗数据

In [41]:
import pandas as pd
import numpy as np
import os
# path=r"D:\案件\山西吕梁网赌\洗钱卡流水"
# file_path=[path+'\\'+i for i in os.listdir(path)]
# file_path2=[i for i in file_path if '$' not in i ]
# file_path2,len(file_path2)

# df_list=[]
# for file_p in file_path2:
#     print(file_p)
#     df=pd.read_excel(file_p ,sheet_name='Sheet1',encoding='utf-8',dtype=object)
#     df['交易户名']=file_p.split('\\')[4]
#     print(len(df.columns))
#     df_list.append(df)
# df=pd.concat(df_list,axis=0)
# df.shape
# df.to_excel(path+"\\"+'洗钱卡汇总.xlsx',index=False,encoding='utf-8')

## 资金分析

In [54]:
import pymysql as ps
#链接数据库
db_host = 'localhost'
db_user = 'root'
db_pwd = '521521'
db_name = 'sxllwd'
char_type = 'utf-8'
db = ps.connect(db_host, db_user, db_pwd, db_name, charset='utf8')
couser = db.cursor()

In [99]:
# 获取交易卡号信息
sql='select * from gas_bank_records  '
couser.execute(sql)
rows = couser.fetchall()
des = couser.description
#获取字段名称
column_name= [item[0] for item in des]
column_name

df=pd.DataFrame(rows,columns=column_name)

In [204]:
df['jyje']=pd.to_numeric(df['jyje'])
df['jyye']=pd.to_numeric(df['jyye'])
df_select=df[['cxkh','jymc','jdbz','jysj','jyje','jyye','jydfmc','jydfzkh','beiz','zysm']]


In [198]:
df_jcz=pd.pivot_table(df_select,index=['cxkh','jymc','jydfzkh','jydfmc'],aggfunc={'jyje':sum,'jysj':[max,min]}).reset_index()

In [200]:
# df_jcz.to_excel('c:/Users/jxh/Desktop/吕梁网贷/整体数据统计.xlsx',index=False,encoding='utf-8')

In [205]:
def jcz_je(row,str1):
    if row['jdbz']==str1:
        return row['jyje']
    else:
        return 0
def jcz_cs(row,str1):
    if row['jdbz']==str1:
        return 1
    else:
        return 0       

In [208]:
df_select['jzje']=df_select.apply(lambda x: jcz_je(x,'进'),axis=1)
df_select['czje']=df_select.apply(lambda x: jcz_je(x,'出'),axis=1)
df_select['jzcs']=df_select.apply(lambda x: jcz_cs(x,'进'),axis=1)
df_select['czcs']=df_select.apply(lambda x: jcz_cs(x,'出'),axis=1)

D:\develop\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\develop\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\develop\anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [237]:
df_tongji=pd.pivot_table(df_select,index=['cxkh','jymc','jydfzkh','jydfmc'],aggfunc={'jyje':sum,'cxkh':len,'czje':sum,'czcs':sum,'jzje':sum,'jzcs':sum,'jysj':[max,min]}).reset_index()

In [238]:
df_tongji.columns=[i+'_'+j for i,j in  df_tongji.columns]

In [239]:
df_tongji.columns=['cxkh', 'jymc', 'jydfzkh','jydfmc','zjycs','czcs','czje', 'zjyje', 'jysj_max', 'jysj_min','jzcs','jzje']
df_tongji.head()

,cxkh,jymc,jydfzkh,jydfmc,zjycs,czcs,czje,zjyje,jysj_max,jysj_min,jzcs,jzje
0,11050172750000000753,北京圣昌人力资源服务有限公司,110727500156273099002000017,企业客户握奇二代网银盾安全产品应付款账户,1,1,45.3,45.3,2019-11-14 14:08:44,2019-11-14 14:08:44,0,0.0
1,11050172750000000753,北京圣昌人力资源服务有限公司,13050164520800000864,邯郸市邯山区俊杭电子设备有限公司,19,0,0.0,640000.0,2019-12-04 15:06:58,2019-12-02 20:35:22,19,640000.0
2,11050172750000000753,北京圣昌人力资源服务有限公司,16023701040007409,郑州郑苏文化传媒有限公司,129,0,0.0,3289700.0,2019-12-06 09:32:16,2019-12-02 18:37:36,129,3289700.0
3,11050172750000000753,北京圣昌人力资源服务有限公司,210401354,汇元银通(北京)在线支付技术有限公司,1,0,0.0,25665.0,2019-11-21 09:18:38,2019-11-21 09:18:38,1,25665.0
4,11050172750000000753,北京圣昌人力资源服务有限公司,33050161782700000875,杭州航秉商贸有限公司,1,0,0.0,21000.0,2019-12-07 00:34:07,2019-12-07 00:34:07,1,21000.0


In [240]:
df_tongji=df_tongji[['cxkh', 'jymc', 'jydfzkh','jydfmc','zjyje','zjycs','czje','czcs','jzje','jzcs', 'jysj_max', 'jysj_min']]
df_tongji.head()

,cxkh,jymc,jydfzkh,jydfmc,zjyje,zjycs,czje,czcs,jzje,jzcs,jysj_max,jysj_min
0,11050172750000000753,北京圣昌人力资源服务有限公司,110727500156273099002000017,企业客户握奇二代网银盾安全产品应付款账户,45.3,1,45.3,1,0.0,0,2019-11-14 14:08:44,2019-11-14 14:08:44
1,11050172750000000753,北京圣昌人力资源服务有限公司,13050164520800000864,邯郸市邯山区俊杭电子设备有限公司,640000.0,19,0.0,0,640000.0,19,2019-12-04 15:06:58,2019-12-02 20:35:22
2,11050172750000000753,北京圣昌人力资源服务有限公司,16023701040007409,郑州郑苏文化传媒有限公司,3289700.0,129,0.0,0,3289700.0,129,2019-12-06 09:32:16,2019-12-02 18:37:36
3,11050172750000000753,北京圣昌人力资源服务有限公司,210401354,汇元银通(北京)在线支付技术有限公司,25665.0,1,0.0,0,25665.0,1,2019-11-21 09:18:38,2019-11-21 09:18:38
4,11050172750000000753,北京圣昌人力资源服务有限公司,33050161782700000875,杭州航秉商贸有限公司,21000.0,1,0.0,0,21000.0,1,2019-12-07 00:34:07,2019-12-07 00:34:07


In [241]:
ydkh=set(df_tongji.cxkh)
# 判断第二层是否调单
df_tongji['y_n_dd']=df_tongji['jydfzkh'].apply(lambda x: 'Y' if x in ydkh else 'N')
df_tongji['jczce']=df_tongji['czje']-df_tongji['jzje']

In [249]:
# 筛选总交易金额>100万，且交易差额>0的
df_100w=df_tongji[(df_tongji['zjyje']>1000000) & (df_tongji['jczce']>0)].sort_values('jczce',ascending=False)

In [251]:
df_100w[df_100w['y_n_dd']=='N'].to_excel("c:/Users/jxh/Desktop/吕梁网贷/交易对手未调单.xlsx",index=False,encoding='utf-8')


In [253]:
df_two=pd.merge(df_100w[df_100w['y_n_dd']=='Y'],df_tongji,left_on='jydfzkh',right_on='cxkh',how='left')

In [258]:
df_two[df_two['jysj_min_x']  <  df_two['jysj_min_y']].shape

(141819, 28)

In [256]:
df_two.to_excel("c:/Users/jxh/Desktop/吕梁网贷/交易对手已经调单.xlsx",index=False,encoding='utf-8')

,cxkh_x,jymc_x,jydfzkh_x,jydfmc_x,zjyje_x,zjycs_x,czje_x,czcs_x,jzje_x,jzcs_x,...,zjyje_y,zjycs_y,czje_y,czcs_y,jzje_y,jzcs_y,jysj_max_y,jysj_min_y,y_n_dd_y,jczce_y
0,6217001830040103551,田昕,6230520700041260673,林志锋,20569824.0,531,20453318.0,528,116506.0,3,...,1.00,1,1.00,1,0.00,0,2019-07-18 11:58:00,2019-07-18 11:58:00,N,1.00
1,6217001830040103551,田昕,6230520700041260673,林志锋,20569824.0,531,20453318.0,528,116506.0,3,...,100000.00,2,100000.00,2,0.00,0,2019-09-14 16:15:41,2019-09-14 16:14:43,N,100000.00
2,6217001830040103551,田昕,6230520700041260673,林志锋,20569824.0,531,20453318.0,528,116506.0,3,...,5240.00,1,0.00,0,5240.00,1,2020-02-10 14:09:34,2020-02-10 14:09:34,N,-5240.00
3,6217001830040103551,田昕,6230520700041260673,林志锋,20569824.0,531,20453318.0,528,116506.0,3,...,3.00,1,3.00,1,0.00,0,2019-08-24 12:34:31,2019-08-24 12:34:31,N,3.00
4,6217001830040103551,田昕,6230520700041260673,林志锋,20569824.0,531,20453318.0,528,116506.0,3,...,5.00,1,5.00,1,0.00,0,2019-08-25 11:09:05,2019-08-25 11:09:05,N,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264014,6217710810977991,田莲英,6222034000025616164,吴少彬,5000795.0,56,2592805.0,29,2407990.0,27,...,40000.00,1,40000.00,1,0.00,0,2020-02-21 14:59:19,2020-02-21 14:59:19,N,40000.00
264015,6217710810977991,田莲英,6222034000025616164,吴少彬,5000795.0,56,2592805.0,29,2407990.0,27,...,3200.00,1,3200.00,1,0.00,0,2020-02-26 17:03:41,2020-02-26 17:03:41,N,3200.00
264016,6217710810977991,田莲英,6222034000025616164,吴少彬,5000795.0,56,2592805.0,29,2407990.0,27,...,8000.00,2,8000.00,2,0.00,0,2020-02-13 23:32:19,2020-02-13 23:32:17,N,8000.00
264017,6217710810977991,田莲英,6222034000025616164,吴少彬,5000795.0,56,2592805.0,29,2407990.0,27,...,57450.00,10,0.00,0,57450.00,10,2020-03-10 11:28:22,2020-03-06 11:02:17,N,-57450.00
